In [1]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModel
from torchvision import models, transforms
from PIL import Image
import faiss
import streamlit as st
import numpy as np

# Load Product Data
products = pd.read_csv("data/raw/products.csv")

# Load User Interactions
interactions = pd.read_csv("data/raw/user_clicks.csv")

# Text Encoder Setup
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
text_model = AutoModel.from_pretrained('bert-base-uncased')
text_model.eval()

# Image Encoder Setup
resnet = models.resnet50(pretrained=True)
resnet.eval()
resnet = nn.Sequential(*list(resnet.children())[:-1])
preprocess_img = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Collaborative Filtering Model
class NCF(nn.Module):
    def __init__(self, num_users, num_items, emb_dim=32):
        super().__init__()
        self.user_emb = nn.Embedding(num_users, emb_dim)
        self.item_emb = nn.Embedding(num_items, emb_dim)
        self.mlp = nn.Sequential(
            nn.Linear(emb_dim*2, 128), nn.ReLU(),
            nn.Linear(128, 1), nn.Sigmoid()
        )
    def forward(self, user_ids, item_ids):
        u = self.user_emb(user_ids)
        v = self.item_emb(item_ids)
        x = torch.cat([u, v], dim=1)
        return self.mlp(x).view(-1)

# Load Pre-trained/Trained CF Model
cf_model = NCF(num_users=1000, num_items=len(products))

# Embedding Computation Functions
def compute_text_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = text_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

def compute_image_embedding(img_path):
    image = Image.open(img_path).convert('RGB')
    image_tensor = preprocess_img(image).unsqueeze(0)
    with torch.no_grad():
        embedding = resnet(image_tensor).view(1, -1)
    return embedding.numpy()

# Precompute Item Embeddings
item_embeddings = []
for idx, row in products.iterrows():
    text_emb = compute_text_embedding(row['title'])
    img_emb = compute_image_embedding(f"data/raw/images/{row['image']}")
    combined_emb = np.concatenate([text_emb.flatten(), img_emb.flatten()])
    item_embeddings.append(combined_emb)
item_embeddings = np.array(item_embeddings).astype('float32')

# Build Faiss Index
faiss.normalize_L2(item_embeddings)
index = faiss.IndexFlatIP(item_embeddings.shape[1])
index.add(item_embeddings)

# Recommendation Functions
def recommend_by_image(image_path, k=5):
    query_emb = compute_image_embedding(image_path)
    faiss.normalize_L2(query_emb)
    D, I = index.search(query_emb, k)
    return products.iloc[I[0]]

def recommend_by_user(user_id, k=5):
    user_tensor = torch.tensor([user_id]*len(products))
    item_tensor = torch.arange(len(products))
    with torch.no_grad():
        scores = cf_model(user_tensor, item_tensor)
    top_indices = torch.topk(scores, k).indices.numpy()
    return products.iloc[top_indices]

# Streamlit App
st.title("Intelligent Multimodal E-commerce Recommender")
user_id = st.number_input("Enter User ID", min_value=0, step=1)
if st.button("Recommend for User"):
    recs = recommend_by_user(user_id)
    st.dataframe(recs[['product_id', 'title']])

uploaded_file = st.file_uploader("Upload Product Image")
if uploaded_file is not None:
    with open("temp_image.jpg", "wb") as f:
        f.write(uploaded_file.getbuffer())
    recs_img = recommend_by_image("temp_image.jpg")
    st.dataframe(recs_img[['product_id', 'title']])


ModuleNotFoundError: No module named 'faiss'

In [2]:
!pip install faiss-cpu
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModel
from torchvision import models, transforms
from PIL import Image
import faiss
import streamlit as st
import numpy as np

# Load Product Data
products = pd.read_csv("data/raw/products.csv")

# Load User Interactions
interactions = pd.read_csv("data/raw/user_clicks.csv")

# Text Encoder Setup
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
text_model = AutoModel.from_pretrained('bert-base-uncased')
text_model.eval()

# Image Encoder Setup
resnet = models.resnet50(pretrained=True)
resnet.eval()
resnet = nn.Sequential(*list(resnet.children())[:-1])
preprocess_img = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Collaborative Filtering Model
class NCF(nn.Module):
    def __init__(self, num_users, num_items, emb_dim=32):
        super().__init__()
        self.user_emb = nn.Embedding(num_users, emb_dim)
        self.item_emb = nn.Embedding(num_items, emb_dim)
        self.mlp = nn.Sequential(
            nn.Linear(emb_dim*2, 128), nn.ReLU(),
            nn.Linear(128, 1), nn.Sigmoid()
        )
    def forward(self, user_ids, item_ids):
        u = self.user_emb(user_ids)
        v = self.item_emb(item_ids)
        x = torch.cat([u, v], dim=1)
        return self.mlp(x).view(-1)

# Load Pre-trained/Trained CF Model
cf_model = NCF(num_users=1000, num_items=len(products))

# Embedding Computation Functions
def compute_text_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = text_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

def compute_image_embedding(img_path):
    image = Image.open(img_path).convert('RGB')
    image_tensor = preprocess_img(image).unsqueeze(0)
    with torch.no_grad():
        embedding = resnet(image_tensor).view(1, -1)
    return embedding.numpy()

# Precompute Item Embeddings
item_embeddings = []
for idx, row in products.iterrows():
    text_emb = compute_text_embedding(row['title'])
    img_emb = compute_image_embedding(f"data/raw/images/{row['image']}")
    combined_emb = np.concatenate([text_emb.flatten(), img_emb.flatten()])
    item_embeddings.append(combined_emb)
item_embeddings = np.array(item_embeddings).astype('float32')

# Build Faiss Index
faiss.normalize_L2(item_embeddings)
index = faiss.IndexFlatIP(item_embeddings.shape[1])
index.add(item_embeddings)

# Recommendation Functions
def recommend_by_image(image_path, k=5):
    query_emb = compute_image_embedding(image_path)
    faiss.normalize_L2(query_emb)
    D, I = index.search(query_emb, k)
    return products.iloc[I[0]]

def recommend_by_user(user_id, k=5):
    user_tensor = torch.tensor([user_id]*len(products))
    item_tensor = torch.arange(len(products))
    with torch.no_grad():
        scores = cf_model(user_tensor, item_tensor)
    top_indices = torch.topk(scores, k).indices.numpy()
    return products.iloc[top_indices]

# Streamlit App
st.title("Intelligent Multimodal E-commerce Recommender")
user_id = st.number_input("Enter User ID", min_value=0, step=1)
if st.button("Recommend for User"):
    recs = recommend_by_user(user_id)
    st.dataframe(recs[['product_id', 'title']])

uploaded_file = st.file_uploader("Upload Product Image")
if uploaded_file is not None:
    with open("temp_image.jpg", "wb") as f:
        f.write(uploaded_file.getbuffer())
    recs_img = recommend_by_image("temp_image.jpg")
    st.dataframe(recs_img[['product_id', 'title']])

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 24.7 MB/s eta 0:00:00


ModuleNotFoundError: No module named 'streamlit'